In [1]:
cd '/home/grad/Desktop/pietro/denovo/2/smartrx/finetuning/mapk1/'

/home/grad/Desktop/pietro/denovo/2/smartrx/finetuning/mapk1


In [2]:
# ============================================================================
# TRAINING: SMARTS-RX -> SMILES (NO SCAFFOLD) - FINE TUNING
# ============================================================================
import os
import sys
import time
import random
import re
import logging
import json
import numpy as np
import tensorflow as tf
from threading import Lock
from typing import List, Tuple

# RDKit Imports
from rdkit import Chem
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogEntry, SmartsMatcher
from sklearn.model_selection import train_test_split

# TensorFlow Imports
from tensorflow.keras.layers import Layer, Dense, Dropout, Embedding, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# ====================================================================
# [ 0. GPU SETUP ]
# ====================================================================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# ====================================================================
# [ 1. CONFIGURATION ]
# ====================================================================

class Config:
    # --- PATHS ---
    SMILES_FILE = "/home/grad/Desktop/pietro/denovo/s4-for-de-novo-drug-design/s4_loro/gen_mio/eval_out_mapk1/train.smi"
    SMARTS_RX_FILE = "/home/grad/Desktop/pietro/denovo/2/SMART_RX/smartsrx-main/smartsrx.json"
    VOCAB_PATH = "/home/grad/Desktop/pietro/denovo/2/smartrx/vocab.json"
    PRETRAINED_MODEL = "/home/grad/Desktop/pietro/denovo/2/smartrx/best_hybrid_model.keras"

    # Hyperparameters
    EMBED_DIM = 512
    LR = 1e-5  # Low learning rate for fine-tuning
    TRANSFORMER_LAYERS = 6
    TRANSFORMER_HEADS = 6
    FF_DIM = 2048
    DROPOUT_RATE = 0.10
    L2_REG = 1e-4
    
    # Training settings
    BATCH_SIZE = 16
    EPOCHS = 5
    MAX_LENGTH = 100
    VALID_RATIO = 0.1
    
    # Curriculum & Augmentation
    CURRICULUM_START_COMPLEXITY = 10
    CURRICULUM_COMPLEXITY_STEP = 1
    LOSS_STABILITY_THRESHOLD = 0.01
    WARMUP_EPOCHS = 5
    AUGMENT_PROB = 0.10
    
    # Generation Monitor
    TEMPERATURE = 1.0
    GEN_NUM = 10
    PRINT_EVERY = 10  
    STEPS_PER_EPOCH = 1000

config = Config()

# Logger Config
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# ====================================================================
# [ 2. CUSTOM LAYERS & LOSS ]
# ====================================================================

def smoothed_loss(y_true, y_pred):
    y_true_int = tf.cast(y_true, tf.int32)
    mask = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true_int, logits=y_pred)
    return tf.reduce_sum(loss * mask) / (tf.reduce_sum(mask) + 1e-9)

class DynamicPositionalEncoding(Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        max_len = config.MAX_LENGTH
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(self.embed_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.embed_dim))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
        super().build(input_shape)
        
    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:, :seq_len, :]
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim}

class ImprovedTransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ffn_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ffn_dim = ffn_dim
        self.rate = rate
        
        # Sub-layers defined in __init__
        self.mha = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim, 
            kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)
        )
        self.ffn = tf.keras.Sequential([
            Dense(ffn_dim, activation="gelu", kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)),
            Dense(embed_dim, kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG))
        ])
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.d1 = Dropout(rate)
        self.d2 = Dropout(rate)

    # --- FIX: ADDED BUILD METHOD TO SUPPRESS WARNINGS ---
    def build(self, input_shape):
        super().build(input_shape)
        # Keras will now correctly track the state of this layer

    def call(self, inputs, training=False, mask=None):
        seq_len = tf.shape(inputs)[1]
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        attn_output = self.mha(inputs, inputs, attention_mask=causal_mask)
        out1 = self.ln1(inputs + self.d1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.ln2(out1 + self.d2(ffn_output, training=training))
        
    def get_config(self):
        return {
            **super().get_config(), 
            "embed_dim": self.embed_dim, 
            "num_heads": self.num_heads, 
            "ffn_dim": self.ffn_dim, 
            "rate": self.rate
        }

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, embed_dim, warmup_steps=10000):
        super().__init__()
        self.embed_dim = tf.cast(embed_dim, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32) + 1e-9
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.embed_dim) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"embed_dim": float(self.embed_dim), "warmup_steps": float(self.warmup_steps)}

# ====================================================================
# [ 3. SMARTS-RX & CHEMICAL HELPERS ]
# ====================================================================

SMARTS_CATALOG = None

def initialize_smarts_catalog():
    global SMARTS_CATALOG
    if SMARTS_CATALOG is not None: return

    try:
        with open(config.SMARTS_RX_FILE, "rt", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        logger.error(f"Error loading SMARTS JSON: {e}")
        sys.exit(1)

    catalog = FilterCatalog()
    count = 0
    for entry in data.get("data", []):
        name = entry.get("specific_type")
        smarts = entry.get("smarts")
        if name and smarts:
            pattern = Chem.MolFromSmarts(smarts)
            if pattern:
                catalog.AddEntry(FilterCatalogEntry(name, SmartsMatcher(pattern)))
                count += 1
    SMARTS_CATALOG = catalog
    logger.info(f"✓ SMARTS-RX catalog loaded: {count} rules.")

def get_smarts_fingerprint(mol: Chem.Mol) -> List[str]:
    if not mol or SMARTS_CATALOG is None: return []
    matches = SMARTS_CATALOG.GetMatches(mol)
    return sorted(list(set([m.GetDescription() for m in matches])))

def robust_tokenize(smiles: str) -> list:
    pattern = (
        r"(\[[^\[\]]{1,6}\]|"                 
        r"Br|Cl|Si|Na|Mg|Mn|Ca|Fe|Zn|Se|Li|K|Al|B|" 
        r"R[0-9]|r[0-9]|a[0-9]|"             
        r"[A-Za-z0-9@+\-\\\/\(\)=#\$\.\%,])"  
    )
    tokens = re.findall(pattern, smiles)
    if tokens.count('[') != tokens.count(']'): return []
    return tokens

def validate_and_fix_smiles(smiles: str) -> str:
    if not smiles: return None
    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is None: return None
        try: Chem.Kekulize(mol, clearAromaticFlags=True)
        except: pass
        return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
    except: return None

def randomize_smiles(smiles: str, num_versions: int = 1) -> List[str]:
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return []
    res = []
    try:
        s = Chem.MolToSmiles(mol, doRandom=True, canonical=False)
        if s: res.append(s)
    except: pass
    return res

def compute_complexity_from_tokens(tokens: List[str]) -> int:
    smiles = ''.join(tokens)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol: return 999
        return Chem.GetSSSR(mol) + smiles.count('(')
    except: return 999

# ====================================================================
# [ 4. DATA PROCESSING ]
# ====================================================================

def process_dataset(smiles_list, vocab):
    initialize_smarts_catalog()
    
    # Prepariamoci a gestire UNK se necessario
    vocab_set = set(vocab)
    processed = []
    
    logger.info(f"Processing {len(smiles_list)} SMILES (Auto-Kekulization ON)...")
    
    for idx, s in enumerate(smiles_list):
        if idx % 5000 == 0 and idx > 0: logger.info(f"Processed {idx}...")
            
        mol = Chem.MolFromSmiles(s)
        if not mol: continue
        
        # --- FIX IMPORTANTE: KEKULIZATION ---
        # Se 'c' manca nel vocab ma 'C' c'è, questo risolve il problema
        try:
            Chem.Kekulize(mol, clearAromaticFlags=True)
        except Exception:
            # Se fallisce kekulizzazione, skippiamo
            continue
            
        # Genera SMILES non isomerico (spesso i vocab vecchi non hanno @)
        target_s = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
        tokens_target = robust_tokenize(target_s)
        
        # SMARTS
        tokens_smarts = get_smarts_fingerprint(mol)
        
        # Qui NON scartiamo più tutto se manca un token.
        # Lasciamo che il generatore usi <UNK> se proprio serve, 
        # ma la Kekulizzazione dovrebbe aver risolto il 99% dei 'c' mancanti.
        
        processed.append((tokens_smarts, tokens_target))
        
    logger.info(f"DONE. Valid data: {len(processed)}")
    return processed

# ====================================================================
# [ 5. CURRICULUM GENERATOR ]
# ====================================================================

class ThreadSafeIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = Lock()
    def __iter__(self): return self
    def __next__(self):
        with self.lock: return next(self.iterator)

def threadsafe_generator(func):
    def wrapper(*args, **kwargs): return ThreadSafeIterator(func(*args, **kwargs))
    return wrapper

class CurriculumSmilesGenerator:
    def __init__(self, processed_data: List[Tuple[List[str], List[str]]], vocab: List[str]):
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.idx2char = {i: c for c, i in self.char2idx.items()}
        
        for token in ['<PAD>', '<START>', '<END>', '<SEP>']:
            if token not in self.char2idx:
                raise ValueError(f"Essential token '{token}' missing in vocab!")
        
        self.original_data = []
        for smarts, target in processed_data:
            comp = compute_complexity_from_tokens(target)
            self.original_data.append(((smarts, target), comp))
        
        valid_comps = [c for _, c in self.original_data if c != 999]
        self.max_complexity = max(valid_comps) if valid_comps else 0
        self.current_complexity = config.CURRICULUM_START_COMPLEXITY
        self.available_data = self._filter_data()
        self.train_smiles = {''.join(target) for (_, target), _ in self.original_data}
        self.lock = Lock()
    
    def _filter_data(self):
        filtered = [dp for dp, c in self.original_data if c <= self.current_complexity]
        return filtered if filtered else [dp for dp, _ in self.original_data]
    
    def update_complexity(self, epoch: int, loss_diff: float = None):
        with self.lock:
            if loss_diff is not None and loss_diff < config.LOSS_STABILITY_THRESHOLD:
                self.current_complexity = min(
                    self.current_complexity + config.CURRICULUM_COMPLEXITY_STEP, 
                    self.max_complexity
                )
            else:
                if epoch <= config.WARMUP_EPOCHS:
                    incr = int((self.max_complexity - config.CURRICULUM_START_COMPLEXITY) * 
                              (epoch / config.WARMUP_EPOCHS))
                    self.current_complexity = config.CURRICULUM_START_COMPLEXITY + incr
                else:
                    self.current_complexity = self.max_complexity
            
            self.available_data = self._filter_data()
            if not self.available_data:
                self.available_data = [dp for dp, _ in self.original_data]
    
    @threadsafe_generator
    def __call__(self):
        PAD_IDX = self.char2idx['<PAD>']
        
        while True:
            inputs = np.full((config.BATCH_SIZE, config.MAX_LENGTH), PAD_IDX, dtype=np.int32)
            targets = np.full_like(inputs, PAD_IDX)
            
            for i in range(config.BATCH_SIZE):
                with self.lock:
                    if not self.available_data:
                        self.available_data = [dp for dp, _ in self.original_data]
                    data_pair = random.choice(self.available_data)
                
                smarts_names, target_tokens = data_pair
                curr_target = target_tokens
                if random.random() < config.AUGMENT_PROB:
                    try:
                        aug = randomize_smiles(''.join(target_tokens), 1)
                        if aug:
                            tok = robust_tokenize(aug[0])
                            if tok and all(t in self.char2idx for t in tok):
                                curr_target = tok
                    except: pass

                seq = (['<START>'] + smarts_names + ['<SEP>'] + curr_target + ['<END>'])
                padded_seq = (seq + ['<PAD>'] * config.MAX_LENGTH)[:config.MAX_LENGTH]
                
                try:
                    indices = [self.char2idx[t] for t in padded_seq]
                except KeyError:
                    seq = (['<START>'] + smarts_names + ['<SEP>'] + target_tokens + ['<END>'])
                    padded_seq = (seq + ['<PAD>'] * config.MAX_LENGTH)[:config.MAX_LENGTH]
                    indices = [self.char2idx[t] for t in padded_seq]

                inputs[i] = indices
                targets[i, :-1] = indices[1:]
                targets[i, -1] = PAD_IDX
                
            yield inputs, targets

    def get_dataset(self):
        return tf.data.Dataset.from_generator(
            self.__call__,
            output_signature=(
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32),
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32)
            )
        ).prefetch(tf.data.AUTOTUNE)

# ====================================================================
# [ 6. MONITORING ]
# ====================================================================

class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        lr = self.model.optimizer.learning_rate
        logs['lr'] = lr(epoch).numpy() if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule) else lr.numpy()
        super().on_epoch_end(epoch, logs)

class EnhancedTrainingMonitor(Callback):
    def __init__(self, val_gen: CurriculumSmilesGenerator):
        super().__init__()
        self.val_gen = val_gen
        self.best_val_loss = np.inf
        self.prev_val_loss = None

    def generate_sample(self, num: int):
        generated, valid = [], []
        PAD = self.val_gen.char2idx['<PAD>']
        START = self.val_gen.char2idx['<START>']
        END = self.val_gen.char2idx['<END>']
        
        input_seq = np.full((1, config.MAX_LENGTH), PAD, dtype=np.int32)
        
        for _ in range(num):
            input_seq.fill(PAD)
            input_seq[0, 0] = START
            
            for t in range(1, config.MAX_LENGTH):
                logits = self.model(input_seq, training=False)[0, t-1]
                logits = logits / config.TEMPERATURE
                probs = tf.nn.softmax(logits).numpy()
                
                sampled = np.random.choice(len(probs), p=probs)
                input_seq[0, t] = sampled
                if sampled == END: break
            
            indices = input_seq[0].tolist()
            raw_tokens = [self.val_gen.idx2char[i] for i in indices if i not in {PAD, START}]
            
            try:
                if '<SEP>' in raw_tokens:
                    target_tokens = raw_tokens[raw_tokens.index('<SEP>') + 1:]
                else:
                    target_tokens = raw_tokens
                
                if '<END>' in target_tokens:
                    target_tokens = target_tokens[:target_tokens.index('<END>')]
                
                smi_str = "".join(target_tokens)
            except Exception:
                smi_str = ""
            
            final = validate_and_fix_smiles(smi_str)
            if final: valid.append(final)
            generated.append(final or smi_str)
            
        return generated, valid

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.prev_val_loss and 'val_loss' in logs:
            diff = (self.prev_val_loss - logs['val_loss']) / (self.prev_val_loss + 1e-9)
            self.val_gen.update_complexity(epoch, diff)
        
        if logs.get('val_loss', np.inf) < self.best_val_loss:
            self.best_val_loss = logs['val_loss']
        
        self.prev_val_loss = logs.get('val_loss')
        
        if (epoch + 1) % config.PRINT_EVERY == 0:
            gen, val = self.generate_sample(config.GEN_NUM)
            validity = len(val) / config.GEN_NUM
            novel = len([s for s in val if s not in self.val_gen.train_smiles])
            logger.info(f"\n--- EPOCH {epoch+1} RESULTS ---")
            logger.info(f"Validity: {validity:.1%} | Novelty: {novel}/{len(val)}")
            if val: logger.info(f"Sample: {val[0]}")



2026-01-30 09:46:08.335049: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-30 09:46:08.350112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769762768.367702 2369230 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769762768.373052 2369230 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769762768.386330 2369230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
# ====================================================================
# [ 7. MAIN EXECUTION ]
# ====================================================================

if __name__ == "__main__":
    # 1. Load Vocabulary
    if not os.path.exists(config.VOCAB_PATH):
        logger.error(f"Vocab file not found: {config.VOCAB_PATH}"); sys.exit(1)
    
    with open(config.VOCAB_PATH, "r") as f:
        vocab = json.load(f)
    
    # 2. Load Model
    logger.info(f"Loading pretrained model from: {config.PRETRAINED_MODEL}")
    try:
        model = load_model(
            config.PRETRAINED_MODEL, 
            custom_objects={
                "DynamicPositionalEncoding": DynamicPositionalEncoding,
                "ImprovedTransformerBlock": ImprovedTransformerBlock,
                "CustomSchedule": CustomSchedule,
                "smoothed_loss": smoothed_loss,
            },
            compile=False # Load without optimizer state first
        )
    except Exception as e:
        logger.error(f"Failed to load model: {e}")
        sys.exit(1)

    # Recompile for Fine-Tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.LR),
        loss=smoothed_loss
    )
    logger.info("Model loaded and recompiled.")

    # 3. Load & Process Data
    if not os.path.exists(config.SMILES_FILE):
        logger.error(f"Smiles file not found: {config.SMILES_FILE}"); sys.exit(1)

    with open(config.SMILES_FILE, "r") as f:
        new_smiles = [l.strip() for l in f if l.strip()]
    
    processed_data = process_dataset(new_smiles, vocab)
    
    if len(processed_data) == 0:
        logger.error("No valid data found after processing. Exiting.")
        sys.exit(1)

    train_data, val_data = train_test_split(processed_data, test_size=config.VALID_RATIO, random_state=42)
    logger.info(f"Train size: {len(train_data)} | Val size: {len(val_data)}")

    # 4. Generators & Callbacks
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)

    run_id = f"ft_run_{int(time.time())}"
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/{run_id}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint(
            filepath="best_finetuned_model.keras", 
            monitor="val_loss", 
            save_best_only=True
        )
    ]

    # 5. Start Training
    logger.info("🔥 Starting Fine-Tuning...")
    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_finetuned_model.keras")
        logger.info("🏆 Fine-Tuning Completed.")
    except KeyboardInterrupt:
        logger.warning("⚠️ Training interrupted.")
        model.save("interrupted_finetuned_model.keras")

2026-01-30 09:46:10,872 [INFO] Loading pretrained model from: /home/grad/Desktop/pietro/denovo/2/smartrx/best_hybrid_model.keras
I0000 00:00:1769762770.951577 2369230 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8729 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:65:00.0, compute capability: 8.6
2026-01-30 09:46:13,376 [INFO] Model loaded and recompiled.
2026-01-30 09:46:13,424 [INFO] ✓ SMARTS-RX catalog loaded: 406 rules.
2026-01-30 09:46:13,425 [INFO] Processing 197 SMILES (Auto-Kekulization ON)...
2026-01-30 09:46:15,395 [INFO] DONE. Valid data: 197
2026-01-30 09:46:15,397 [INFO] Train size: 177 | Val size: 20
2026-01-30 09:46:15,433 [INFO] 🔥 Starting Fine-Tuning...


Epoch 1/5


I0000 00:00:1769762786.247560 2369551 service.cc:152] XLA service 0x712548004170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769762786.247575 2369551 service.cc:160]   StreamExecutor device (0): NVIDIA RTX A2000 12GB, Compute Capability 8.6
2026-01-30 09:46:27.170415: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769762789.133384 2369551 cuda_dnn.cc:529] Loaded cuDNN version 90300
2026-01-30 09:46:32.392183: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_154', 240 bytes spill stores, 240 bytes spill loads

2026-01-30 09:46:33.019997: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_154', 

   1/1000 ━━━━━━━━━━━━━━━━━━━━ 14:02:29 51s/step - loss: 0.8287

I0000 00:00:1769762826.292490 2369551 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 161s 111ms/step - loss: 0.5282 - val_loss: 0.7020 - lr: 1.0000e-05
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 106s 106ms/step - loss: 0.2261 - val_loss: 0.9922 - lr: 1.0000e-05
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 106s 106ms/step - loss: 0.1932 - val_loss: 0.9358 - lr: 1.0000e-05
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 106s 106ms/step - loss: 0.1842 - val_loss: 1.0000 - lr: 1.0000e-05
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 106s 106ms/step - loss: 0.1786 - val_loss: 0.9529 - lr: 1.0000e-05


2026-01-30 09:56:04,859 [INFO] 🏆 Fine-Tuning Completed.
